<a href="https://colab.research.google.com/github/KittLao/sakila/blob/main/sakila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 37.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [3]:
import mysql.connector


In [13]:
# Database different from database identifier. Identifier contains multiple
# databases.
USER = "admin"
PASSWORD = "21223032315684kK"
HOST = "sakila-mysql.cdfvrj0gw8cb.us-east-1.rds.amazonaws.com"
DATABASE = "sakila"

cnx = mysql.connector.connect(user=USER, password=PASSWORD,
                              host=HOST,
                              database=DATABASE)



In [16]:
# Need to use buffered cursor so we can execute initialize cursor again if needed.
cursor = cnx.cursor(buffered=True)

# Tables

In [33]:
show_table_query = ("show tables")
cursor.execute(show_table_query)
tables = cursor.fetchall()
print(tables)

[('actor',), ('actor_info',), ('address',), ('category',), ('city',), ('country',), ('customer',), ('customer_list',), ('film',), ('film_actor',), ('film_category',), ('film_list',), ('film_text',), ('inventory',), ('language',), ('nicer_but_slower_film_list',), ('payment',), ('rental',), ('sales_by_film_category',), ('sales_by_store',), ('staff',), ('staff_list',), ('store',)]


# How much data


In [50]:
# fetchall gets all records from previous executed query.
# Just use f strings.

def record_count_per_table(cursor, table):
  query = f"select count(*) from {table}"
  cursor.execute(query)
  result = cursor.fetchall()
  return result[0][0]

for table in tables:
  table = table[0]
  count = record_count_per_table(cursor, table)
  print(table, count)


actor 200
actor_info 200
address 603
category 16
city 600
country 109
customer 599
customer_list 599
film 1000
film_actor 5462
film_category 1000
film_list 1000
film_text 0
inventory 4581
language 6
nicer_but_slower_film_list 1000
payment 16044
rental 16044
sales_by_film_category 16
sales_by_store 2
staff 2
staff_list 2
store 2


# Columns for Table

In [52]:
def record_count_per_table(cursor, table):
  query = f"show columns from {table}"
  cursor.execute(query)
  result = cursor.fetchall()
  return result

for table in tables:
  table = table[0]
  columns = record_count_per_table(cursor, table)
  print(table, columns)


actor [('actor_id', b'smallint unsigned', 'NO', 'PRI', None, 'auto_increment'), ('first_name', b'varchar(45)', 'NO', '', None, ''), ('last_name', b'varchar(45)', 'NO', 'MUL', None, ''), ('last_update', b'timestamp', 'NO', '', b'CURRENT_TIMESTAMP', 'DEFAULT_GENERATED on update CURRENT_TIMESTAMP')]
actor_info [('actor_id', b'smallint unsigned', 'NO', '', b'0', ''), ('first_name', b'varchar(45)', 'NO', '', None, ''), ('last_name', b'varchar(45)', 'NO', '', None, ''), ('film_info', b'text', 'YES', '', None, '')]
address [('address_id', b'smallint unsigned', 'NO', 'PRI', None, 'auto_increment'), ('address', b'varchar(50)', 'NO', '', None, ''), ('address2', b'varchar(50)', 'YES', '', None, ''), ('district', b'varchar(20)', 'NO', '', None, ''), ('city_id', b'smallint unsigned', 'NO', 'MUL', None, ''), ('postal_code', b'varchar(10)', 'YES', '', None, ''), ('phone', b'varchar(20)', 'NO', '', None, ''), ('location', b'geometry', 'NO', 'MUL', None, ''), ('last_update', b'timestamp', 'NO', '', b'C

# Time period for Payments

In [53]:
time_payment_query = ("select min(payment_date) as first, max(payment_date) as last from payment")
cursor.execute(time_payment_query)
result =  cursor.fetchall()
print(result)

[(datetime.datetime(2005, 5, 24, 22, 53, 30), datetime.datetime(2006, 2, 14, 15, 16, 3))]


# Where do events occer? How many are from these locations?


*   District
*   Address
*   City
*   Postal
*   Country





In [60]:
location_query = """
select
  a.address as address,
  a.district as district,
  a.postal_code as postal_code,
  ci.city as city,
  co.country as country
from address as a
inner join city as ci
on a.city_id = ci.city_id
inner join country as co
on ci.country_id = co.country_id
"""
cursor.execute(location_query)
result = cursor.fetchmany(5)
for record in result:
  print(record)

location_analysis_query = f"""
with locations as (
  {location_query}
)
select
	count(address) as address_count,
	count(district) as district_count,
	count(postal_code) as postal_code_count,
	count(city) as city_count,
	count(country) as country_count
from locations
group by address, district, postal_code, city, country
"""

cursor.execute(location_analysis_query)
result = cursor.fetchmany(5)
for record in result:
  print(record)


('1168 Najafabad Parkway', 'Kabol', '40301', 'Kabul', 'Afghanistan')
('1924 Shimonoseki Drive', 'Batna', '52625', 'Batna', 'Algeria')
('1031 Daugavpils Parkway', 'Béchar', '59025', 'Béchar', 'Algeria')
('757 Rustenburg Avenue', 'Skikda', '89668', 'Skikda', 'Algeria')
('1892 Nabereznyje Telny Lane', 'Tutuila', '28396', 'Tafuna', 'American Samoa')
(1, 1, 1, 1, 1)
(1, 1, 1, 1, 1)
(1, 1, 1, 1, 1)
(1, 1, 1, 1, 1)
(1, 1, 1, 1, 1)


# Movie for every Payment

In [61]:
movie_per_payment_query = """
select
	p.payment_id as payment,
	f.title as movie
from payment as p
inner join rental as r on p.rental_id = r.rental_id
inner join inventory as i on r.inventory_id = i.inventory_id
inner join film as f on i.film_id = f.film_id
"""

cursor.execute(movie_per_payment_query)
for record in cursor.fetchmany(5):
  print(record)

(11630, 'ACADEMY DINOSAUR')
(13956, 'ACADEMY DINOSAUR')
(7578, 'ACADEMY DINOSAUR')
(11124, 'ACADEMY DINOSAUR')
(4607, 'ACADEMY DINOSAUR')


# Sum movie revenue

In [62]:
revenue_per_movie_query = """
select
	f.title as movie,
	sum(p.amount) as total_revenue
from payment as p
inner join rental as r on p.rental_id = r.rental_id
inner join inventory as i on r.inventory_id = i.inventory_id
inner join film as f on i.film_id = f.film_id
group by movie
order by total_revenue desc
"""

cursor.execute(revenue_per_movie_query)
for record in cursor.fetchmany(5):
  print(record)

('TELEGRAPH VOYAGE', Decimal('231.73'))
('WIFE TURN', Decimal('223.69'))
('ZORRO ARK', Decimal('214.69'))
('GOODFELLAS SALUTE', Decimal('209.69'))
('SATURDAY LAMBS', Decimal('204.72'))


# Top grossing cities


In [65]:
revenue_per_city_query = """
select ci.city as city, sum(p.amount) as revenue
from payment as p
inner join customer as cu on p.customer_id = cu.customer_id
inner join address as a on a.address_id = cu.address_id
inner join city as ci on a.city_id = ci.city_id
group by  ci.city_id
order by revenue desc
"""

cursor.execute(revenue_per_city_query)
for record in cursor.fetchmany(5):
  print(record)

('Cape Coral', Decimal('221.55'))
('Saint-Denis', Decimal('216.54'))
('Aurora', Decimal('198.50'))
('Molodetšno', Decimal('195.58'))
('Apeldoorn', Decimal('194.61'))


# Revenue of movie per city, customer and month

In [66]:
movie_revenue_per_city_customer_month_query = """
select
	f.title as title,
	ci.city as city_name,
	CONCAT_WS(' ', cu.first_name, cu.last_name) as customer_name,
	DATE_FORMAT(p.payment_date, "%Y-%M") as month_,
	sum(p.amount) as revenue
from payment as p
inner join rental as r on p.rental_id = r.rental_id
inner join inventory as i on r.rental_id = i.inventory_id
inner join film as f on i.film_id = f.film_id
inner join customer as cu on p.customer_id = cu.customer_id
inner join address as a on a.address_id = cu.address_id
inner join city as ci on a.city_id = ci.city_id
group by title, city_name, customer_name, month_
order by month_ asc, revenue desc
"""

cursor.execute(movie_revenue_per_city_customer_month_query)
for record in cursor.fetchmany(5):
  print(record)

('SMOKING BARBARELLA', 'Joliet', 'NATHANIEL ADAM', '2005-July', Decimal('16.98'))
('SMOKING BARBARELLA', 'Phnom Penh', 'ALLISON STANLEY', '2005-July', Decimal('15.98'))
('WEDDING APOLLO', 'al-Qatif', 'TRAVIS ESTEP', '2005-July', Decimal('12.98'))
('WAR NOTTING', 'Juiz de Fora', 'KENT ARSENAULT', '2005-July', Decimal('11.99'))
('SUNRISE LEAGUE', 'Székesfehérvár', 'TERRANCE ROUSH', '2005-July', Decimal('11.99'))
